In [ ]:
import pickle

import matplotlib.pyplot as plt
import healpy as hp
import numpy as np
import os

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

In [ ]:
current_dir = os.getcwd()

In [ ]:
# data = pickle.load(open(current_dir + "/parametric_d0_UWB_test_invN/maps/after_changes_nonoise_seed1_None_k0.pkl", "rb"))
data = pickle.load(open(current_dir + "/../FMM/test_merge/maps/test_None.pkl", "rb"))

In [ ]:
fwhm_in = data["fwhm_in"]
fwhm_out = data["fwhm_out"]
fwhm_rec = data["fwhm_rec"]

# comp_maps_input = data["components_in"]
# comp_maps_convolved_recon = data["components_convolved_recon"]
# comp_maps_rec = data["components_iter"]
comp_maps_input = data["maps_in"]
# comp_maps_convolved_recon = data["components_convolved_recon"]
comp_maps_rec = data["maps"][:4]
comp_maps_res = comp_maps_input - comp_maps_rec
center = data["center"]
# seenpix = data["seenpix"]
convergence = data["convergence"][0]

In [ ]:
print(convergence)

In [ ]:
plt.plot(convergence)
plt.xlabel("Iteration")
plt.ylabel("Convergence")
plt.title("PCG Convergence")
plt.yscale("log")
plt.grid()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

ax1.plot(data["TOD_qubic"])
ax1.set_xlabel("Time")
ax1.set_ylabel("TOD")
ax1.set_title("TOD QUBIC")

ax2.plot(data["TOD_external"])
ax2.set_xlabel("Time")
ax2.set_ylabel("TOD")
ax2.set_title("TOD Planck")

plt.tight_layout()

In [ ]:
print("FWHM in: ", fwhm_in)
print("mean fwhm in: ", np.mean(fwhm_in))
print("FWHM out: ", fwhm_out)
print("FWHM rec: ", fwhm_rec)
mean_fwhm = np.mean(fwhm_in)
print("FWHM mean: ", mean_fwhm)

In [ ]:
fwhm_rec[0]

In [ ]:
comp_maps_test = np.zeros(np.shape(comp_maps_input))
# C_test = HealpixConvolutionGaussianOperator(fwhm=mean_fwhm)
# C_test = HealpixConvolutionGaussianOperator(fwhm=0)

# for i, icomp in enumerate(comp_maps_input):
#     C_test = HealpixConvolutionGaussianOperator(fwhm=fwhm_rec[0])
#     comp_maps_test[i] = C_test(icomp)
#     # comp_maps_test[i] = comp_maps_convolved_recon[i]
#     comp_maps_res[i] =  comp_maps_test[i] - comp_maps_rec[i]


for i, icomp in enumerate(comp_maps_input):
    fwhm_test = np.mean(fwhm_in[i*2 : (i + 1)*2])
    print(fwhm_test, fwhm_rec[i])
    # fwhm_test = fwhm_rec[i]
    comp_maps_test[i] = np.mean([HealpixConvolutionGaussianOperator(fwhm=fwhm_in[j])(comp_maps_input[j]) for j in range(i*2, (i + 1)*2)], axis=0)
    # C_test = HealpixConvolutionGaussianOperator(fwhm=fwhm_test)
    # comp_maps_test[i] = C_test(icomp)
    # comp_maps_test[i] = comp_maps_convolved_recon[i]
    comp_maps_res[i] =  comp_maps_test[i] - comp_maps_rec[i]

name_comp = ["CMB", "Dust"]
STOKES = ["I", "Q", "U"]

for imap in [comp_maps_input, comp_maps_rec, comp_maps_res]:
    imap[:, ~seenpix, :] = hp.UNSEEN

istk = 0
n = 2
sigma_input = np.std(comp_maps_input[:, seenpix, istk], axis=0)
sigma_rec = np.std(comp_maps_rec[:, seenpix, istk], axis=0)
sigma_res = np.std(comp_maps_rec[:, seenpix, istk] - comp_maps_input[:, seenpix, istk], axis=0)

reso = 20

k = 0

plt.figure(figsize=(20, 10))
for icomp in range(2):
    # hp.gnomview((comp_maps_input[icomp, :, istk]), rot=center,
    hp.gnomview((comp_maps_test[icomp, :, istk]), rot=center,
                        reso=reso,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} input",  
                        cmap="jet",
                        # min = - n * sigma_input[icomp],
                        # max = n * sigma_input[icomp],
                        sub=(2, 3, k + 1))
    
    hp.gnomview((comp_maps_rec[icomp, :, istk]), rot=center,
                        reso=reso,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} Output",  
                        cmap="jet",
                        # min = - n * sigma_rec[icomp],
                        # max = n * sigma_rec[icomp],
                        sub=(2, 3, k + 2))
    
    hp.gnomview(comp_maps_res[icomp, :, istk], rot=center,
                        reso=reso,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} Residual",  
                        cmap="jet",
                        # min = - n * sigma_res[icomp],
                        # max = n * sigma_res[icomp],
                        sub=(2, 3, k + 3))
    k += 3

plt.savefig("figures/test_maps.png")
plt.show()

In [ ]:
4128768/196608

In [ ]:
stop

In [ ]:
icomp = 0

test_in = data["components_in"][icomp].copy()
test_out = data["components_iter"][icomp].copy()

test_in[~seenpix, :] = 0
test_out[~seenpix, :] = 0

In [ ]:
def chi2_fwhm(fwhm):
    if fwhm < 0 or fwhm > 0.1:
        return np.inf
    else:
        C = HealpixConvolutionGaussianOperator(fwhm)
        input_maps = C(test_in)
            
        return np.sum((test_out - input_maps) ** 2) / np.sum(input_maps ** 2)

In [ ]:
list_chi2 = []

fwhm = np.logspace(np.log10(0.0005), np.log10(0.05), num=80)
for i in fwhm:
    list_chi2.append(chi2_fwhm(i))

list_chi2 = np.array(list_chi2)

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(fwhm, list_chi2)
plt.vlines(fwhm[np.where(list_chi2 == np.min(list_chi2))[0][0]], 0, 2, color="red", linestyle="--", label=f"Best fit : {fwhm[np.where(list_chi2 == np.min(list_chi2))[0][0]].round(4)}")
plt.vlines(fwhm_rec[icomp], 0, 2, color="green", linestyle="--", label=f"FWHM rec : {fwhm_rec[icomp].round(4)}")
plt.title("Chi2 as a function of fwhm")
plt.xlabel("FWHM")
plt.ylabel("Chi2")
plt.grid()
plt.legend()
plt.show()